In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  1.13 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse
os.environ["DATASET"] = "../datasets"

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from hdecoder.BaseModel import BaseModel
from hdecoder import build_model
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt

/home/djjin/Mygit/X-Decoder


Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
from utils.arguments import load_vcoco_opt_command, load_vcoco_parser

cmdline_args = load_vcoco_parser()
cmdline_args.conf_files = [os.path.join(home_dir, "configs/hdecoder/vcoco.yaml")]

model_path = '../data/output/test/00089100/default/raw_model_states.pt'
cmdline_args.overrides = ['DONT_LOAD_MODEL', 'false', 'PYLEARN_MODEL', model_path] 

opt = load_vcoco_opt_command(cmdline_args)
opt = init_distributed(opt)

In [4]:
print(opt['base_path'])
print(opt["RESUME"])

../
False


In [5]:
from trainer import HDecoder_Trainer as Trainer
if cmdline_args.user_dir:
    absolute_user_dir = os.path.abspath(cmdline_args.user_dir)
    opt['base_path'] = absolute_user_dir
trainer = Trainer(opt)
trainer.eval()

INFO:trainer.distributed_trainer:Setting SAVE_DIR as data/output/test
INFO:trainer.distributed_trainer:Using CUDA
INFO:trainer.distributed_trainer:Save config file to data/output/test/conf_copy.yaml
INFO:trainer.distributed_trainer:Base learning rate: 0.0001
INFO:trainer.distributed_trainer:Number of GPUs: 1
INFO:trainer.distributed_trainer:Gradient accumulation steps: 1
INFO:trainer.default_trainer:Imported base_dir at base_path ../
INFO:trainer.default_trainer:Pipeline for training: HDecoderPipeline
INFO:trainer.default_trainer:-----------------------------------------------
INFO:trainer.default_trainer:Evaluating model ... 
INFO:base_dir.pipeline.HDecoderPipeline:CDNHOI(
  (backbone): D2FocalNet(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        (bloc

Evaluation start ...


INFO:detectron2.data.common:Serializing 4946 elements to byte tensors and concatenating them all ...
INFO:detectron2.data.common:Serialized dataset takes 3.69 MiB
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 11/2473. Dataloading: 0.1034 s/iter. Inference: 0.4816 s/iter. Eval: 0.0013 s/iter. Total: 0.5863 s/iter. ETA=0:24:03
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 20/2473. Dataloading: 0.0859 s/iter. Inference: 0.4807 s/iter. Eval: 0.0015 s/iter. Total: 0.5683 s/iter. ETA=0:23:14
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 29/2473. Dataloading: 0.0823 s/iter. Inference: 0.4816 s/iter. Eval: 0.0013 s/iter. Total: 0.5658 s/iter. ETA=0:23:02
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 39/2473. Dataloading: 0.0682 s/iter. Inference: 0.4811 s/iter. Eval: 0.0011 s/iter. Total: 0.5509 s/iter. ETA=0:22:20
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 49/2473. Dataloading: 0.0603 s/iter. Inference: 0.4815 s/iter. Eval: 0.0010 s/iter. Total:

------------------------------------------------------------
               hold_obj: #GTs = 3608, AP = 0.1372
                  stand: #GTs = 4118, AP = 0.5446
              sit_instr: #GTs = 1916, AP = 0.2500
             ride_instr: #GTs = 0556, AP = 0.1327
                   walk: #GTs = 0597, AP = 0.2402
               look_obj: #GTs = 3347, AP = 0.1159
              hit_instr: #GTs = 0349, AP = 0.0177
                hit_obj: #GTs = 0349, AP = 0.2360
                eat_obj: #GTs = 0521, AP = 0.2390
              eat_instr: #GTs = 0521, AP = 0.6010
             jump_instr: #GTs = 0635, AP = 0.1913
              lay_instr: #GTs = 0387, AP = 0.3289
    talk_on_phone_instr: #GTs = 0285, AP = 0.1017
              carry_obj: #GTs = 0472, AP = 0.1138
              throw_obj: #GTs = 0244, AP = 0.0265
              catch_obj: #GTs = 0246, AP = 0.0121
              cut_instr: #GTs = 0269, AP = 0.0465
                cut_obj: #GTs = 0269, AP = 0.2535
                    run: #GTs = 0687, A

INFO:trainer.default_trainer:{'vcoco_val/vcoco': {'AP_hold_obj': 0.13720505068818348, 'AP_stand': 0.5446213234397418, 'AP_sit_instr': 0.2500329011634043, 'AP_ride_instr': 0.13268171046272628, 'AP_walk': 0.2402128412508539, 'AP_look_obj': 0.11590145811398322, 'AP_hit_instr': 0.017744378089550532, 'AP_hit_obj': 0.23601558192896133, 'AP_eat_obj': 0.23899265935844424, 'AP_eat_instr': 0.6010053453029219, 'AP_jump_instr': 0.19125173963277112, 'AP_lay_instr': 0.3288826483516966, 'AP_talk_on_phone_instr': 0.1017181858613769, 'AP_carry_obj': 0.11376401838228342, 'AP_throw_obj': 0.026523249714054745, 'AP_catch_obj': 0.012100259291270527, 'AP_cut_instr': 0.046546918829667175, 'AP_cut_obj': 0.25348648755798203, 'AP_run': 0.5151156075768426, 'AP_work_on_computer_instr': 0.2780225058311563, 'AP_ski_instr': 0.11460309604333362, 'AP_surf_instr': 0.10701007746462292, 'AP_skateboard_instr': 0.10890864136986861, 'AP_smile': 0.6167012046166761, 'AP_drink_instr': 0.026769705199094346, 'AP_kick_obj': 0.0546

{'vcoco_val/vcoco': {'AP_hold_obj': 0.13720505068818348,
  'AP_stand': 0.5446213234397418,
  'AP_sit_instr': 0.2500329011634043,
  'AP_ride_instr': 0.13268171046272628,
  'AP_walk': 0.2402128412508539,
  'AP_look_obj': 0.11590145811398322,
  'AP_hit_instr': 0.017744378089550532,
  'AP_hit_obj': 0.23601558192896133,
  'AP_eat_obj': 0.23899265935844424,
  'AP_eat_instr': 0.6010053453029219,
  'AP_jump_instr': 0.19125173963277112,
  'AP_lay_instr': 0.3288826483516966,
  'AP_talk_on_phone_instr': 0.1017181858613769,
  'AP_carry_obj': 0.11376401838228342,
  'AP_throw_obj': 0.026523249714054745,
  'AP_catch_obj': 0.012100259291270527,
  'AP_cut_instr': 0.046546918829667175,
  'AP_cut_obj': 0.25348648755798203,
  'AP_run': 0.5151156075768426,
  'AP_work_on_computer_instr': 0.2780225058311563,
  'AP_ski_instr': 0.11460309604333362,
  'AP_surf_instr': 0.10701007746462292,
  'AP_skateboard_instr': 0.10890864136986861,
  'AP_smile': 0.6167012046166761,
  'AP_drink_instr': 0.026769705199094346,
  